In [0]:
#@title ⬅️ Run This Cell for VNC
Ngrok_Token = "1Q2OdVvHqZXIMXCbYRpM60PrsE7_7oe4deNjJvKAmDjPDAVQD" #@param {type:"string"}
Root_Password = "war014rior" #@param {type:"string"}
import random, string, urllib.request, json, getpass, os, time, IPython, uuid
import ipywidgets as widgets
from IPython.display import HTML, clear_output

password = Root_Password

loadingBtn = widgets.Button(description = "Installing",
                          disabled = True,
                          button_style = 'warning',
                          tooltip = "Installing",
                          icon = 'check')
display(loadingBtn)

if not os.path.exists("/usr/bin/xfce4-session"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && sudo apt update && sudo apt install xfce4 xfce4-goodies")
  
if not os.path.exists("/usr/bin/tightvncserver"):
  get_ipython().system_raw("sudo apt install tightvncserver")

if not os.path.exists("/usr/bin/firefox"):
  get_ipython().system_raw("sudo apt install firefox")

if not os.path.exists("/root/.Ngrok"):
  ! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -P /root/.Ngrok
  ! unzip -qq -n /root/.Ngrok/ngrok-stable-linux-amd64.zip -d /root/.Ngrok

! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc
with open('xstartup','w') as p:
  p.write('''

#!/bin/sh

xrdb $HOME/.Xresources
xsetroot -solid grey
#x-terminal-emulator -geometry 80x24+10+10 -ls -title "$VNCDESKTOP Desktop" &
#x-window-manager &
# Fix to make GNOME work
/etc/X11/Xsession
export XKL_XMODMAP_DISABLE=1
exec /usr/bin/startxfce4 &

''')
! mkdir -p ~/.vnc
! mv xstartup ~/.vnc/xstartup
! sudo chmod +x ~/.vnc/xstartup
! wget -q -c --no-check-certificate --content-disposition https://github.com/K-E-N-W-A-Y/VNC-Script-for-Colab/raw/master/JDownloader.jar
! wget -q -c https://github.com/tonikelope/megabasterd/releases/download/v6.31/MegaBasterd_6.31.jar
! mv /content/*.jar ~/
! sudo chmod +x ~/*.jar
! mkdir -p ~/JDownloader2
! mv ~/JDownloader.jar ~/JDownloader2/JDownloader.jar
! mkdir -p /content/JDownloader
! mkdir -p /content/Megabasterd
! mkdir -p /content/QBittorrent

get_ipython().system_raw('/usr/sbin/sshd -D &')

authtoken = Ngrok_Token

get_ipython().system_raw('/root/.Ngrok/ngrok authtoken $authtoken && /root/.Ngrok/ngrok tcp 22 &')

time.sleep(5)
try:
  with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
    data = json.loads(response.read().decode())
    (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
    clear_output()
    print(f'SSH command: ssh -L 5901:127.0.0.1:5901 -p{port} root@{host}')

    print(f'Root password: {password}')
except:
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error, Rerun the Cell.</h2><br></center>"))

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Check VM's Status</h3>
Loop_Check = True #@param {type:"boolean"}
Loop_Interval = 5 #@param {type:"slider", min:1, max:15, step:1}
# ================================================================ #

import time
from IPython.display import clear_output
Loop = True

try:
  while Loop == True:
    clear_output(wait=True)
    !top -bcn1 -w512
    if Loop_Check == False:
      Loop = False
    else:
      time.sleep(Loop_Interval)
except:
  clear_output(wait=True)

In [0]:
#@markdown <h3>⬅️ Install Cloud Commander (file manager)</h3>
#@markdown <br><center><img src='https://minormole.github.io/RcloneLab/img/title_cloud_commander.png' height="60" alt="netdata"/></center>

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue
  
def AutoSSH(name,port):
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_AutoSSH_CC():
  if CheckProcess("cloudcmd", "") != True:
    get_ipython().system_raw("cloudcmd --online --no-auth --show-config --show-file-name --editor 'deepword' --packer 'tar' --port $Port_CC --progress --no-confirm-copy --confirm-move --name 'RcloneLab File Manager' --keys-panel --no-contact --console --sync-console-path --no-terminal --no-vim --columns 'name-size-date' --no-log &")
  if CheckProcess("autossh", Random_URL_CC) != True:
    AutoSSH(Random_URL_CC, Port_CC)

def Start_Localhost_CC():
  try:
    clear_output(wait=True)
    !autossh -l $Random_URL_CC -M 0 -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:$Port_CC ssh.localhost.run
  except:
    Control_Panel_CC()
  Control_Panel_CC()

def Control_Panel_CC():
  clear_output(wait=True)
  display(MakeLabel("✔ Successfully", "success"), MakeButton("Recheck", Start_AutoSSH_CC), MakeButton("Backup Website", Start_Localhost_CC), HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">File Manager</h2><h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">" \
                    "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_URL_CC + ".serveo.net/fs/content/\" target=\"_blank\">Website</a></h3>"))

try:
  try:
    Random_URL_CC
  except NameError:
    Random_URL_CC = "cc" + RandomGenerator()
    Port_CC = "7007"
    display(MakeLabel('Installing in Progress', 'warning'))
  if os.path.isfile("/tools/node/bin/cloudcmd") == False:
    get_ipython().system_raw("rm -rf /content/sample_data/ && npm i -g npm && npm i cloudcmd -g")
  if os.path.isfile("/usr/bin/autossh") == False:
    get_ipython().system_raw("apt update -qq -y && apt install autossh -qq -y")
  Start_AutoSSH_CC()
  Control_Panel_CC()
except:
  clear_output(wait=True)
  display(MakeLabel("✘ Unsuccessfully", "danger")) 

In [0]:
#@markdown <h3>⬅️ Click Here to Mount Google Drive</h3>
#@markdown <h4>⚠️ If you download the file directly to Google Drive, it might have data corruption, use this at your own risk!</h4>
#@markdown <h4>✅ The rclone method is recommended and always the safest.</h4>

import os, uuid; import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import drive, output

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)
    
def DemountForm():
  clear_output(wait=True)
  display(MakeLabel("Demount Confirm?", ""), MakeButton("Confirm", Demount, "danger"), MakeButton("Cancel", ShowGD, "warning"))
    
def Demount():
  clear_output(wait=True)
  get_ipython().system_raw("pkill -9 -e -f drive")
  !rm -rf /root/.config/Google/DriveFS/
  !echo "If you want to mount Google Drive run this cell again."
  
def ShowGD():
  try:
    RefreshPath()
  except:
    clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), MakeButton("Demount", DemountForm, "danger"))

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output(wait=True)
    
ShowGD()

In [0]:
#@markdown <h3>⬅️ Click Here to Install JDownloader</h3>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Dropdown(value="/content", options=["/content", "/content/Downloads"], description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output(wait=True)
  Email.value = ""
  Password.value = ""
  Device.value = ""
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath, MakeButton("Refresh", RefreshPath, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))
  
def RestartForm():
  clear_output(wait=True)
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))
  
def ExitForm():
  clear_output(wait=True)
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output(wait=True)
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()
  
def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output(wait=True)
  display(MakeButton("Start", Restart, "info"))
  
def Show():
  clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can login to the WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))
  
    
if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output(wait=True)
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-8-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader/libs && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h && wget -q https://minormole.github.io/RcloneLab/res/jdownloader/sevenzipjbinding1509.jar -O /root/.JDownloader/libs/sevenzipjbinding1509.jar && wget -q https://minormole.github.io/RcloneLab/res/jdownloader/sevenzipjbinding1509Linux.jar -O /root/.JDownloader/libs/sevenzipjbinding1509Linux.jar")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:  
  Show()